In [7]:
import os
os.environ["PATH"] += os.pathsep + 'C:/Program Files/Graphviz/bin'

from diagrams import Diagram, Edge, Cluster
from diagrams.aws.compute import EC2
from diagrams.aws.database import RDS
from diagrams.generic.database import SQL
from diagrams.aws.network import ELB
from diagrams.aws.integration import SQS
from  diagrams.aws.storage import S3
from diagrams.saas.chat import Slack
from diagrams.custom import Custom

attributes = {"pad": "1.0", "fontsize": "25"}
with Diagram("Sample data flow in microdervice architecture", show=True, direction="LR",
             outformat="png",
             graph_attr=attributes):
    
    with Cluster("Service layer") as services:
        service1 = EC2('Service1')
        service2 = EC2('Service2')
        service3 = EC2('service3')
        request_handlers = [service1, service2, service3]
        
    with Cluster("Elastic load balancers") as elbs:
        elb1 =  ELB("Load balancer1")
        elb2 =  ELB("Load balancer2")
        elbs = [elb1, elb2]
    
    with Cluster("Database layer") as dblayer:
        rds_dbs = [RDS('db1'), RDS('db2')]
        sql_db = SQL('db3')
    
    with Cluster('Batch processing layer') as batch_processors:
        service4 = EC2('Service4')
        service5 = EC2('Service5')
        batch_processors = [service4, service5]
    with Cluster('Notification channels') as notifications:
        email = Custom('Email notification', 'email.png')
        sms = Custom('SMS notification', 'sms.png')
        notifications = [Slack('slack notification'), email, sms]

    
    elb1 >>  Edge(label='requests') >> request_handlers
    elb2 >>  Edge(label='requests') >> request_handlers
    service1 >> Edge(label='store data') >> rds_dbs
    [service2, service3] >> Edge(label='store data') >> sql_db
    request_handlers >> SQS('sqs') >> batch_processors 
    service4 >> Edge(label='sending error notification', color='red', style='dashed') >> notifications
    service5 >> Edge(label='sending error notification', color='red', style='dashed') >> notifications